# Семинар 6

Для тех, кому интересно векторное дифференцирование и почему в линейной регрессии оптимальная оценка весов равна $w = \left(X^TX\right)^{-1}X^Ty$:

- http://www.machinelearning.ru/wiki/images/5/50/MOMO17_Seminar2.pdf
- https://github.com/esokolov/ml-course-hse/blob/master/2017-fall/seminars/sem02-linregr-part1.pdf

# Теория

https://github.com/esokolov/ml-minor-hse/blob/master/colloquium-2017/colloquium_minor_problems_linear.pdf

Задачи 1, 3, 5

# Practice

Мы поработаем с данными о сообществах в США. Описание датасета:

http://archive.ics.uci.edu/ml/datasets/communities+and+crime

Датасет на кэггле (в формате .csv):

https://www.kaggle.com/kkanda/communities%20and%20crime%20unnormalized%20data%20set

Будем предсказывать количество насильственных преступлений относительно численности населения.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings

warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('crimedata.csv', na_values=["?"])
# будем работать не со всеми колонками
requiredColumns = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]
data = data[data.columns[requiredColumns]]
# некоторые значения целевой переменной пропущены
X = data.loc[data['ViolentCrimesPerPop'].notnull(), :].drop('ViolentCrimesPerPop', axis=1)
y = data['ViolentCrimesPerPop'][X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### 1 Baseline

Обучим линейную регрессию и выведем качество по метрике MSE на обучающей и тестовой выборке.

In [ ]:
lr = LinearRegression().fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test))))

Добавим регуляризатор и посмотрим, изменилось ли качество. В качестве метода регуляризации используем Ridge ($L_2$-регуляризация).

In [ ]:
ridge = Ridge(5.0).fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test))))

### 2 Scaling

А что изменится при нормировании признаков? Попробуем MinMaxScaler. Есть ли разница? Влияет ли нормирование на качество линейной регрессии? А на качество регуляризованной? Почему так?

In [ ]:
sc = MinMaxScaler()
X_train_scaled = pd.DataFrame(data=sc.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=sc.transform(X_test), columns=X_test.columns)

In [ ]:
lr = LinearRegression().fit(X_train_scaled,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_scaled))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_scaled))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_scaled,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_scaled))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_scaled))))

### 3 High/low variance

Полезны ли признаки, имеющие высокую дисперсию? А низкую?

In [ ]:
features_variance = X_train_scaled.var().sort_values(ascending=False)
features_variance

Попробуем удалить признаки с самой низкой дисперсией и посмотреть, как изменится качество. В `sklearn` есть специальный инструмент для такого наивного отбора признаков. Стоит ли нормализовать перед этим признаки?

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# можно убрать все признаки, вариативность которых меньше заданного значения
vs_transformer = VarianceThreshold(0.01)
X_train_var = pd.DataFrame(data=vs_transformer.fit_transform(X_train_scaled), columns=X_train_scaled.columns[vs_transformer.get_support()])
X_test_var = pd.DataFrame(data=vs_transformer.transform(X_test_scaled), columns=X_test_scaled.columns[vs_transformer.get_support()])

X_train_var.shape

In [ ]:
lr = LinearRegression().fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_var))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_var))))

### 4 Correlation

Можно выбрать k признаков, которые дают наиболее высокие значения корреляции с целевой переменной.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Выбираем 15 лучших признаков
sb = SelectKBest(f_regression, k=15)

X_train_kbest = pd.DataFrame(data=sb.fit_transform(X_train_var, y_train), columns=X_train_var.columns[sb.get_support()])
X_test_kbest = pd.DataFrame(data=sb.transform(X_test_var), columns=X_test_var.columns[sb.get_support()])

In [ ]:
lr = LinearRegression().fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_kbest))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_kbest))))

А можно выбрать самые значимые признаки с точки зрения регрессии с $L_1$-регуляризацией.

In [ ]:
from sklearn.feature_selection import SelectFromModel

lasso = Lasso(5.0)
l1_select = SelectFromModel(lasso)

X_train_l1 = pd.DataFrame(data=l1_select.fit_transform(X_train_var, y_train), columns=X_train_var.columns[l1_select.get_support()])
X_test_l1 = pd.DataFrame(data=l1_select.transform(X_test_var), columns=X_test_var.columns[l1_select.get_support()])

X_train_l1.shape

In [ ]:
lr = LinearRegression().fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_l1))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_l1))))

### 5 Pipeline

А можно сделать все вышеописанное сразу:

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('variance', VarianceThreshold(0.01)),
    ('selection', SelectFromModel(Lasso(5.0))),
    ('regressor', Ridge(5.0))
])

pipe.fit(X_train, y_train)

pipe.named_steps

In [ ]:
print ("Train: {}".format(mean_squared_error(y_train, pipe.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe.predict(X_test))))

Можно также настраивать параметры с помощью `GridSearch`:

In [ ]:
pipe.get_params()

In [ ]:
param_grid = {
    'variance__threshold': [0.005, 0.0075, 0.009, 0.01, 0.011, 0.012],
    'selection__estimator__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
    'regressor__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
}
grid_search = GridSearchCV(pipe, param_grid, iid=False, cv=5)

grid_search.fit(X_train, y_train)

In [ ]:
pipe_best = grid_search.best_estimator_
pipe_best.named_steps

In [ ]:
pipe_best.fit(X_train, y_train)
print ("Train: {}".format(mean_squared_error(y_train, pipe_best.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe_best.predict(X_test))))